## Imports 

In [50]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score, mean_squared_error, max_error, mean_absolute_error
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel


In [51]:
def printAvalStat(truth, preds):
    print(" The RVE is: ", explained_variance_score(truth, preds))
    print(" The rmse is: ", mean_squared_error(truth, preds, squared=False))
    corr, pval=pearsonr(truth, preds)
    print(" The Correlation Score is: %6.4f (p-value=%e)\n"%(corr,pval))

    print(" The Maximum Error is: ", max_error(truth, preds))
    print(" The Mean Absolute Error is:", mean_absolute_error(truth, preds),"\n")

def nfold_valid(X_train_Valids, y_train, mdl):
    kf = KFold(n_splits=5, shuffle=True, random_state=23)
    kf.get_n_splits(X_train_Valids)
    TRUTH_nfold=None
    PREDS_nfold=None
    for train_index, test_index in kf.split(X_train_Valids):
        X_train_nfold, X_ivs_nfold = X_train_Valids[train_index], X_train_Valids[test_index]
        y_train_nfold, y_ivs_nfold = y_train[train_index], y_train[test_index]

        mdl.fit(X_train_nfold, y_train_nfold)
        preds = mdl.predict(X_ivs_nfold)
        if TRUTH_nfold is None:
            PREDS_nfold=preds
            TRUTH_nfold=y_ivs_nfold
        else:
            PREDS_nfold=np.hstack((PREDS_nfold, preds))
            TRUTH_nfold=np.hstack((TRUTH_nfold, y_ivs_nfold))
        
    printAvalStat(TRUTH_nfold, PREDS_nfold)

#print("N-fold cross validation of nXf_train, after sequencial reduction feature selection")
#nfold_valid(nXf_train)

### Loading and understanding the Dataset

In [52]:
X_train, X_ivs, y_train, col_names = pickle.load(open("drd2_data.pickle", "rb"))
print(X_train.shape)

(7337, 2132)


In [53]:
print(X_ivs.shape)

(816, 2132)


In [54]:
print(y_train.shape)

(7337,)


In [55]:
N,M=X_train.shape
v=np.hstack((y_train.reshape((N,1)), X_train))
pd.DataFrame(v)

,0,1,2,3,4,5,6,7,8,9,...,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132
0,0.654947,541.280138,541.656,10.0,1.0,8.0,1.0,10.0,40.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.649995,426.197714,426.582,5.0,1.0,9.0,1.0,4.0,30.0,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.154947,348.183778,348.446,4.0,0.0,3.0,0.0,3.0,26.0,50.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.616176,1455.763803,1456.831,27.0,19.0,23.0,17.0,16.0,105.0,206.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,0.359725,387.151368,387.886,4.0,0.0,4.0,0.0,4.0,27.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7332,0.000000,467.149047,467.513,6.0,0.0,6.0,0.0,5.0,32.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7333,0.002193,240.162649,240.350,2.0,0.0,3.0,0.0,2.0,18.0,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7334,0.293481,510.317874,510.802,4.0,0.0,10.0,0.0,4.0,37.0,79.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7335,0.596804,393.187483,393.556,4.0,2.0,5.0,1.0,5.0,28.0,55.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
pd.DataFrame(np.corrcoef(v.T))

,0,1,2,3,4,5,6,7,8,9,...,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132
0,1.000000,0.160182,0.160216,0.130117,0.084939,0.151968,0.095929,0.126690,0.157545,0.165189,...,0.011360,0.095138,0.078900,-0.021728,-0.012977,0.001865,0.012468,0.031884,-0.027774,0.058370
1,0.160182,1.000000,0.999998,0.919389,0.772557,0.888252,0.784495,0.856242,0.992766,0.973177,...,0.213406,0.019559,0.127229,0.117371,-0.017119,0.053576,0.212881,0.342831,0.173077,0.219727
2,0.160216,0.999998,1.000000,0.919128,0.772379,0.888072,0.784320,0.855980,0.992623,0.972956,...,0.213435,0.019583,0.127309,0.117351,-0.017201,0.053498,0.212889,0.342785,0.173016,0.219646
3,0.130117,0.919389,0.919128,1.000000,0.838103,0.863232,0.843652,0.931528,0.922469,0.905146,...,0.196534,-0.008557,0.095891,0.064926,-0.026235,0.043314,0.181134,0.332638,0.137266,0.205303
4,0.084939,0.772557,0.772379,0.838103,1.000000,0.704868,0.993448,0.653117,0.774714,0.769215,...,0.185163,-0.032325,0.083849,0.022735,-0.013150,0.045537,0.110449,0.409722,0.210139,0.231743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,0.001865,0.053576,0.053498,0.043314,0.045537,0.036037,0.047958,0.038387,0.059813,0.061989,...,0.018420,-0.020652,-0.018922,-0.009945,-0.009870,1.000000,0.063520,0.047193,0.032638,0.027846
2129,0.012468,0.212881,0.212889,0.181134,0.110449,0.254921,0.115041,0.202605,0.213051,0.218212,...,0.022700,-0.015847,0.075295,-0.020981,-0.004391,0.063520,1.000000,0.026101,0.049729,0.000372
2130,0.031884,0.342831,0.342785,0.332638,0.409722,0.276091,0.410854,0.233237,0.342548,0.342609,...,0.006494,0.017914,0.009569,-0.004214,-0.020504,0.047193,0.026101,1.000000,0.123067,0.127682
2131,-0.027774,0.173077,0.173016,0.137266,0.210139,0.177572,0.216401,0.081396,0.184924,0.190152,...,0.004409,-0.026145,0.009127,0.051670,-0.029027,0.032638,0.049729,0.123067,1.000000,0.082630


In [57]:
def features_to_remove_with_threshold(threshold):
    corr_matrix = pd.DataFrame(np.corrcoef(v.T))
    selected_values_lines = [(row, value) for row, value in enumerate(corr_matrix.iloc[1:, 0]) if abs(value) < threshold]

    # print(len(selected_values_lines))
    # Display the selected values and lines
    # for row, value in selected_values_lines:
    #     print(f"Row: {row}, Value: {value}")

    features_to_remove = []
    for values, line_number in enumerate(selected_values_lines):
        #print(f"Feature {line_number}: {values}")
        features_to_remove.append(line_number[0])

    return features_to_remove

In [58]:
%%script false --no-raise-error
threshold = np.arange(0.05, 0.16, 0.02)

for i in threshold:
    features_to_remove = features_to_remove_with_threshold(i)
    X_train_cut= np.delete(X_train, features_to_remove, 1)

    print(f"N-fold cross validation of X_train for {X_train_cut.shape[1]}")
    print("Random Forest")
    RFR_mdl = RandomForestRegressor(n_estimators=10, random_state=0, min_samples_leaf=3, max_depth = 8, n_jobs=6)
    nfold_valid(X_train_cut, y_train, RFR_mdl)

    print("Linear Regressor")
    LR_mdl = LinearRegression(n_jobs=6)
    nfold_valid(X_train_cut, y_train, LR_mdl)  

    print("Decision Tree")
    DTR_mdl = DecisionTreeRegressor(max_depth=5)
    nfold_valid(X_train_cut, y_train, DTR_mdl)

    print("KNN")
    KNN_mdl = KNeighborsRegressor(n_jobs=6)
    nfold_valid(X_train_cut, y_train, KNN_mdl)

    print("SVR")
    SVR_mdl = SVR(kernel='rbf', C=10, gamma=0.1)
    nfold_valid(X_train_cut, y_train, SVR_mdl)

    print("MLP")
    MLP_mdl = MLPRegressor()
    nfold_valid(X_train_cut, y_train, MLP_mdl)


Couldn't find program: 'false'


### Data Treatement, scaling data and looking to features with most correlation

In [59]:
sScaler = StandardScaler()
sScaler.fit(X_train)
X_train_scaled = sScaler.transform(X_train)
X_ivs_scaled = sScaler.transform(X_ivs)

pT_scaler = PowerTransformer()
pT_scaler.fit(X_train)
X_train_power_t = pT_scaler.transform(X_train)
X_ivs_power_t = pT_scaler.transform(X_train)

v_scaled=np.hstack((y_train.reshape((N,1)), X_train_scaled))
pd.DataFrame(v_scaled)

c:\Users\joaom\Documents\FCUL\1ºSemestre\Aprendizagem Automática\Home assignments\.env\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\joaom\Documents\FCUL\1ºSemestre\Aprendizagem Automática\Home assignments\.env\Lib\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


,0,1,2,3,4,5,6,7,8,9,...,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132
0,0.654947,0.609921,0.608648,1.098414,-0.081854,0.246156,-0.058057,1.908209,0.737657,0.614996,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
1,0.649995,-0.023790,-0.024513,-0.103427,-0.081854,0.413306,-0.058057,-0.243126,-0.038629,0.032423,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
2,0.154947,-0.453381,-0.454433,-0.343795,-0.477479,-0.589590,-0.508809,-0.601682,-0.349144,-0.355960,...,1.234467,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
3,0.616176,5.645607,5.644129,5.184672,7.039406,2.753396,7.153978,4.059544,5.783518,5.702803,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,5.029661,3.091413,5.116060
4,0.359725,-0.238802,-0.237426,-0.343795,-0.477479,-0.422441,-0.508809,-0.243126,-0.271515,-0.355960,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7332,0.000000,0.201712,0.200698,0.136941,-0.477479,-0.088142,-0.508809,0.115430,0.116628,-0.122930,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
7333,0.002193,-1.048209,-1.049199,-0.824531,-0.477479,-0.589590,-0.508809,-0.960238,-0.970173,-0.822018,...,-0.810066,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,3.091413,-0.195463
7334,0.293481,0.439425,0.438883,-0.343795,-0.477479,0.580455,-0.508809,-0.243126,0.504771,0.770349,...,-0.810066,-0.195463,-0.179087,3.210153,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463
7335,0.596804,-0.205564,-0.206229,-0.343795,0.313772,-0.255292,-0.058057,0.115430,-0.193886,-0.161769,...,1.234467,-0.195463,-0.179087,-0.311512,-0.168687,-0.105656,-0.136394,-0.198821,-0.323477,-0.195463


In [60]:
pd.DataFrame(np.corrcoef(v_scaled.T))


,0,1,2,3,4,5,6,7,8,9,...,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132
0,1.000000,0.160182,0.160216,0.130117,0.084939,0.151968,0.095929,0.126690,0.157545,0.165189,...,0.011360,0.095138,0.078900,-0.021728,-0.012977,0.001865,0.012468,0.031884,-0.027774,0.058370
1,0.160182,1.000000,0.999998,0.919389,0.772557,0.888252,0.784495,0.856242,0.992766,0.973177,...,0.213406,0.019559,0.127229,0.117371,-0.017119,0.053576,0.212881,0.342831,0.173077,0.219727
2,0.160216,0.999998,1.000000,0.919128,0.772379,0.888072,0.784320,0.855980,0.992623,0.972956,...,0.213435,0.019583,0.127309,0.117351,-0.017201,0.053498,0.212889,0.342785,0.173016,0.219646
3,0.130117,0.919389,0.919128,1.000000,0.838103,0.863232,0.843652,0.931528,0.922469,0.905146,...,0.196534,-0.008557,0.095891,0.064926,-0.026235,0.043314,0.181134,0.332638,0.137266,0.205303
4,0.084939,0.772557,0.772379,0.838103,1.000000,0.704868,0.993448,0.653117,0.774714,0.769215,...,0.185163,-0.032325,0.083849,0.022735,-0.013150,0.045537,0.110449,0.409722,0.210139,0.231743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,0.001865,0.053576,0.053498,0.043314,0.045537,0.036037,0.047958,0.038387,0.059813,0.061989,...,0.018420,-0.020652,-0.018922,-0.009945,-0.009870,1.000000,0.063520,0.047193,0.032638,0.027846
2129,0.012468,0.212881,0.212889,0.181134,0.110449,0.254921,0.115041,0.202605,0.213051,0.218212,...,0.022700,-0.015847,0.075295,-0.020981,-0.004391,0.063520,1.000000,0.026101,0.049729,0.000372
2130,0.031884,0.342831,0.342785,0.332638,0.409722,0.276091,0.410854,0.233237,0.342548,0.342609,...,0.006494,0.017914,0.009569,-0.004214,-0.020504,0.047193,0.026101,1.000000,0.123067,0.127682
2131,-0.027774,0.173077,0.173016,0.137266,0.210139,0.177572,0.216401,0.081396,0.184924,0.190152,...,0.004409,-0.026145,0.009127,0.051670,-0.029027,0.032638,0.049729,0.123067,1.000000,0.082630


In [61]:
%%script false --no-raise-error
threshold = np.arange(0.05, 0.16, 0.02)

for i in threshold:
    features_to_remove = features_to_remove_with_threshold(i)
    X_train_cut= np.delete(X_train_scaled, features_to_remove, 1)

    print(f"N-fold cross validation of X_train for {X_train_cut.shape[1]}")
    print("Random Forest")
    RFR_mdl = RandomForestRegressor(n_estimators=10, random_state=0, min_samples_leaf=3, max_depth = 8, n_jobs=6)
    nfold_valid(X_train_cut, y_train, RFR_mdl)

    print("Linear Regressor")
    LR_mdl = LinearRegression(n_jobs=6)
    nfold_valid(X_train_cut, y_train, LR_mdl)  

    print("Decision Tree")
    DTR_mdl = DecisionTreeRegressor(max_depth=5)
    nfold_valid(X_train_cut, y_train, DTR_mdl)

    print("KNN")
    KNN_mdl = KNeighborsRegressor(n_jobs=6)
    nfold_valid(X_train_cut, y_train, KNN_mdl)

    print("SVR")
    SVR_mdl = SVR()
    nfold_valid(X_train_cut, y_train, SVR_mdl)

    # print("MLP")
    # MLP_mdl = MLPRegressor()
    # nfold_valid(X_train_cut, y_train, MLP_mdl)

N-fold cross validation of X_train for 2132
Random Forest
 The RVE is:  0.43083638196535623
 The rmse is:  0.208703513879559
 The Correlation Score is: 0.6608 (p-value=0.000000e+00)

 The Maximum Error is:  0.8645408990344696
 The Mean Absolute Error is: 0.1652836905537773 

Linear Regressor
 The RVE is:  0.4270934090524995
 The rmse is:  0.2094028980287027
 The Correlation Score is: 0.7004 (p-value=0.000000e+00)

 The Maximum Error is:  1.0548827375254515
 The Mean Absolute Error is: 0.15847652396570752 

Decision Tree
 The RVE is:  0.2859254385612089
 The rmse is:  0.23376700864205158
 The Correlation Score is: 0.5359 (p-value=0.000000e+00)

 The Maximum Error is:  0.8883078161666667
 The Mean Absolute Error is: 0.18556654545452797 

KNN
 The RVE is:  0.5169615407967393
 The rmse is:  0.19592788060403094
 The Correlation Score is: 0.7213 (p-value=0.000000e+00)

 The Maximum Error is:  0.8476752418
 The Mean Absolute Error is: 0.1433638550345373 

SVR
 The RVE is:  0.6195917293083877


### Selecting features by droping the features with correlation and by selection process

In [62]:
def featureSelect(x_train, n_features, t):
    rfr=RandomForestRegressor(random_state=0, n_jobs=6)
    rfr.fit(x_train, y_train)

    
    sel = SelectFromModel(estimator=rfr, threshold= t) #Change the threshold! See what happens!
    sel.fit(x_train, y_train)
    
    
    features=sel.get_support()
    Features_selected =np.arange(n_features)[features]
    print("The features selected are columns: ", Features_selected,".\n Number of features:", len(Features_selected))
    
    X_train_rffs=sel.transform(x_train)
    return X_train_rffs

In [63]:
%%script false --no-raise-error

threshold = np.arange(0.05, 0.16, 0.02)

for i in threshold:
    features_to_remove = features_to_remove_with_threshold(i)
    X_train_cut= np.delete(X_train_scaled, features_to_remove, 1)
    X_train_rffs = featureSelect(X_train_cut, X_train_cut.shape[1], .003)

    print(f"\nN-fold cross validation of X_train for {X_train_cut.shape[1]} and {i} as threshold value")
    print("Random Forest")
    RFR_mdl = RandomForestRegressor(n_estimators=10, random_state=0, min_samples_leaf=3, max_depth = 8, n_jobs=6)
    nfold_valid(X_train_rffs, y_train, RFR_mdl)

    print("Linear Regressor")
    LR_mdl = LinearRegression(n_jobs=6)
    nfold_valid(X_train_rffs, y_train, LR_mdl)  

    print("Decision Tree")
    DTR_mdl = DecisionTreeRegressor(max_depth=5)
    nfold_valid(X_train_rffs, y_train, DTR_mdl)

    print("KNN")
    KNN_mdl = KNeighborsRegressor(n_jobs=6)
    nfold_valid(X_train_rffs, y_train, KNN_mdl)
    
    print("SVR")
    SVR_mdl = SVR()
    nfold_valid(X_train_rffs, y_train, SVR_mdl)

Couldn't find program: 'false'


### Principal Component analizys (PCA)

In [64]:
def pca(x_train, n_comps):
    pca = PCA(n_components=n_comps)
    pca.fit(x_train)
    tve=0
    for i, ve in enumerate(pca.explained_variance_ratio_):
        tve+=ve
        print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve) )
    print()
    # print("Actual Eigenvalues:", pca.singular_values_)
    # for i,comp in enumerate(pca.components_):
    #    print("PC",i, "-->", comp)   
    X_train_pca = pca.transform(x_train) 
    return X_train_pca

def kpca(x_train, n_comps):
    kpca = KernelPCA(n_components=n_comps, kernel='rbf', n_jobs=4)#, gamma=3)
    kpca.fit(x_train)
    X_train_kpca = kpca.transform(x_train)
    return X_train_kpca

In [72]:
threshold = np.arange(0.05, 0.16, 0.02)

n_components_pca = [30, 60, 100]

# for t in threshold:
    
#     features_to_remove = features_to_remove_with_threshold(t)
#     X_train_cut= np.delete(X_train_scaled, features_to_remove, 1)
#     X_train_rffs = featureSelect(X_train_cut, X_train_cut.shape[1], .003)
    
#     for i in n_components_pca:
        # pca = pca(X_train_scaled, i) 
        # X_train_pca = pca.transform(X_train_scaled)

N,M = X_train_scaled.shape

features_to_remove = features_to_remove_with_threshold(0.03)
X_train_cut= np.delete(X_train_scaled, features_to_remove, 1)
# X_train_rffs = featureSelect(X_train_cut, X_train_cut.shape[1], .003)
# X_train_rffs = featureSelect(X_train_scaled, M, .003)
X_train_pca = pca(X_train_cut, 250)


print(f"\nN-fold cross validation of X_train for {X_train_cut.shape[1]}")
print("Random Forest")
RFR_mdl = RandomForestRegressor(n_estimators=10, random_state=0, min_samples_leaf=3, max_depth = 8, n_jobs=6)
nfold_valid(X_train_pca, y_train, RFR_mdl)

# print("Linear Regressor")
# LR_mdl = LinearRegression(n_jobs=6)
# nfold_valid(X_train_pca, y_train, LR_mdl)  

# print("Decision Tree")
# DTR_mdl = DecisionTreeRegressor(max_depth=5)
# nfold_valid(X_train_pca, y_train, DTR_mdl)

print("KNN")
KNN_mdl = KNeighborsRegressor(n_jobs=6)
nfold_valid(X_train_pca, y_train, KNN_mdl)

print("SVR")
SVR_mdl = SVR()
nfold_valid(X_train_pca, y_train, SVR_mdl)

PC0 - Variance explained:  0.0395 - Total Variance:  0.0395
PC1 - Variance explained:  0.0212 - Total Variance:  0.0606
PC2 - Variance explained:  0.0197 - Total Variance:  0.0803
PC3 - Variance explained:  0.0170 - Total Variance:  0.0973
PC4 - Variance explained:  0.0124 - Total Variance:  0.1096
PC5 - Variance explained:  0.0121 - Total Variance:  0.1218
PC6 - Variance explained:  0.0117 - Total Variance:  0.1334
PC7 - Variance explained:  0.0109 - Total Variance:  0.1443
PC8 - Variance explained:  0.0105 - Total Variance:  0.1548
PC9 - Variance explained:  0.0101 - Total Variance:  0.1649
PC10 - Variance explained:  0.0095 - Total Variance:  0.1745
PC11 - Variance explained:  0.0091 - Total Variance:  0.1836
PC12 - Variance explained:  0.0085 - Total Variance:  0.1921
PC13 - Variance explained:  0.0077 - Total Variance:  0.1998
PC14 - Variance explained:  0.0076 - Total Variance:  0.2073
PC15 - Variance explained:  0.0072 - Total Variance:  0.2145
PC16 - Variance explained:  0.0071